# Improved interpretability for computer-aided severity assesment of Rethinopathy of Prematurity 
paper to appear as oral presentation at SPIE2019 Medical Imaging, Computer-aided diagnosis track 

In [3]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import h5py
import sys
sys.path.append('./iMIMIC-RCVs/')
sys.path.append('./iMIMIC-RCVs/rcv_utils.py')
sys.path.append('./iMIMIC-RCVs/scripts/keras_vis_rcv/')
sys.path.append('./iMIMIC-RCVs/scripts/keras_vis_rcv/vis/')

import rcv_utils
import keras
import keras.backend as K
import sklearn.model_selection
import sklearn.linear_model
from rop_utils import *
PWD=''

/home/mara/venv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 1. Loading concept measures from handcrafted features, for each image in the dataset

In [13]:
#feat = pd.read_csv('./sample_5kFeatures.csv')
feat = pd.read_csv('../5kFeatures.csv')
feature_names = feat.columns
feat['Image Name'] = feat['Image Name'].astype(str).str[:-4]
feat=feat.set_index('Image Name')
concepts = ['cti median', 'cti mean', 'curvature median', 'curvature mean', 'Avg segment diameter median', 'Avg point diameter mean']

In [14]:
meas = feat.to_dict('index')
# checking that the features have been successfully loaded
concept_measure = 'cti median'
image_file = 'BEAU-0071_000387_2_Posterior_os_No'
print  '{} measure for input file {} = '.format(concept_measure, image_file), meas[image_file][concept_measure]

cti median measure for input file BEAU-0071_000387_2_Posterior_os_No =  1.023713797


### 2. Loading ROP data

In [6]:
# Note: we cannot share the original images. 
# For more information you can contact me at: 
# mara.graziani@hevs.ch

h5file = '../split0.data/train.h5'
images, labels, original_files, classes = load_rop_data(h5file)
test_images, test_labels, test_original_files, test_classes = load_rop_data('../split0.data/test.h5')
inputs=np.array([swap(sample) for sample in images[:]])
print inputs.shape
test_inputs = np.array([swap(sample) for sample in test_images[:]])
print test_inputs.shape

data
labels
original_files
data
labels
original_files
(3264, 224, 224, 3)
(988, 224, 224, 3)


### 3. Loading the model

In [7]:
# Note: we cannot share the model weights. 
# For more information you can contact me at: 
# mara.graziani@hevs.ch

model = keras.models.load_model('../split0/final_model.h5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_7x7_conv (Conv2D)     (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_7x7_bn (BatchNormaliz (None, 112, 112, 64) 192         Conv2d_1a_7x7_conv[0][0]         
__________________________________________________________________________________________________
Conv2d_1a_7x7_act (Activation)  (None, 112, 112, 64) 0           Conv2d_1a_7x7_bn[0][0]           
__________________________________________________________________________________________________
MaxPool_2a

In [8]:
model.load_weights('../split0/best_weights.h5')

### 4. Computing the RCVs
For more information about the method you can read the paper
on regression concept vectors for bidirectional explanations
(https://github.com/medgift/iMIMIC-RCVs).

In [10]:
layers = ['Conv2d_2b_1x1_conv', 
          'Conv2d_2c_3x3_conv', 
          'Mixed_3b_Concatenated', 
          'Mixed_3c_Concatenated',
          'Mixed_4b_Concatenated',
          'Mixed_4c_Concatenated',
          'Mixed_5c_Concatenated'] 
max_rep = 3
batch_size=32
for repetition in range(0, 1):
    for l in layers[:]:
        feats=[]
        i=0
        if os.path.isfile('../rcv/phis/'+str(repetition)+'_concepts_phis_'+l+'.npy'):
            print '../rcv/phis/'+str(repetition)+'_concepts_phis_'+l+' already esists'
            continue
        get_layer_output = K.function([model.layers[0].input],
                                  [model.get_layer(l).output])
        while i+batch_size<len(inputs):
            feats.append(get_layer_output([inputs[i:i+batch_size]]))
            i+=batch_size
        feats.append(get_layer_output([inputs[i:]]))
        feats=np.array(feats)
        feats=feats.reshape((feats.shape[0]*feats.shape[1]*feats.shape[2], feats.shape[3], feats.shape[4], feats.shape[5]))
        np.save('../rcv/phis/'+str(repetition)+'_concepts_phis_'+l, np.asarray(feats))

../rcv/phis/0_concepts_phis_Conv2d_2b_1x1_conv already esists
../rcv/phis/0_concepts_phis_Conv2d_2c_3x3_conv already esists
../rcv/phis/0_concepts_phis_Mixed_3b_Concatenated already esists
../rcv/phis/0_concepts_phis_Mixed_3c_Concatenated already esists
../rcv/phis/0_concepts_phis_Mixed_4b_Concatenated already esists
../rcv/phis/0_concepts_phis_Mixed_4c_Concatenated already esists
../rcv/phis/0_concepts_phis_Mixed_5c_Concatenated already esists


In [15]:
l = layer = layers[-2]
repetition=0
rop_class = 'Plus'
rcvs = {}
acts = np.load('../rcv/phis/'+str(repetition)+'_concepts_phis_'+l+'.npy')
l=layer = layers[-1]
repetition=0
print layers
rop_class = 'Plus'
rcvs = {}
print 'Extracting RCV at layer: ', layer
acts = np.load('../rcv/phis/'+str(repetition)+'_concepts_phis_'+l+'.npy')
for c in concepts[:1]:
    if c not in rcvs.keys():
        rcvs[c]={}
    print 'Analysing concept: ', c
    fvec=[]
    to_keep=[]
    classes=[]
    fvec, to_keep = get_concept_measures_vector(meas, original_files, c)
    print acts.shape
    X=(np.asarray([acts[i].ravel() for i in to_keep], dtype=np.float64))  
    print X.shape
    classes = [get_sample_class(labels, i) for i in to_keep]
    idxs = get_class_indices(classes, rop_class)
    print idxs
    print X[idxs].shape
    reg_score, cv = solve_regression(
                np.squeeze(X[idxs], axis=1), 
                np.squeeze(fvec[idxs], axis=1), 
                random_state=repetition
    )
    print 'Regression output: ', reg_score, ' (', rop_class, ')'
    rcvs[c][rop_class] = cv

['Conv2d_2b_1x1_conv', 'Conv2d_2c_3x3_conv', 'Mixed_3b_Concatenated', 'Mixed_3c_Concatenated', 'Mixed_4b_Concatenated', 'Mixed_4c_Concatenated', 'Mixed_5c_Concatenated']
Extracting RCV at layer:  Mixed_5c_Concatenated
Analysing concept:  cti median
(3264, 7, 7, 1024)
(3238, 50176)
[[2158]
 [2159]
 [2160]
 ...
 [3235]
 [3236]
 [3237]]
(1080, 1, 50176)
N.  0 ..
720
0.5435025503141853
N.  1 ..
720
0.45235606547785745
N.  2 ..
720
0.5343546518571483
0.510071089216397
angle:  0.9955726934551511
angle:  0.992904898020826
Regression output:  0.510071089216397  ( Plus )


In [ ]:
regression_outputs = compute_regression(max_rep, layers[2:], meas, original_files, acts, labels, concepts)

Layer:  Mixed_3b_Concatenated
(3264, 28, 28, 256)
N.  0 ..
716
0.05323439828445953
N.  1 ..
716
-7.720960726701713e+19
N.  2 ..
716
-6.416333465403717e+18
-2.7875313577473618e+19
angle:  1.573347931489505
angle:  1.5854454860461016
N.  0 ..
722
-0.1294199935423268
N.  1 ..
723


In [23]:
try:
    os.path.exist('./results')
except:
    os.mkdir('./results')
np.save('./results/preliminary_regression_outputs_def', regression_outputs)

NameError: name 'regression_outputs' is not defined

In [ ]:
plot_dynamics(regression_outputs, 'Normal', concepts, layers)

In [ ]:
plot_dynamics(regression_outputs, 'Plus', concepts, layers)

In [ ]:
### 5. Computing the sensitivity scores
## to run
import utils
reload(utils)
from utils import * 
batch_size=32
rop_class='Normal'
layer=-1
#del rcvs_plus
#rcvs_plus=np.load('./preliminary_rcvs_plus.npy').item()
rcvs_plus=np.load('./preliminary_rcvs_normal.npy').item() #np.load('./preliminary_rcvs_plus.npy').item()
class_beginning_index = get_class_indices(test_classes, rop_class)[0][0]
print class_beginning_index
for concept in concepts:
    index = class_beginning_index
    '''
    while index + batch_size < len(test_inputs):
        print 'in'
        compute_sensitivity_scores(concept, 
                                   rcvs_plus[concept][rop_class][0], 
                                   model, 
                                   layers[layer], 
                                   rop_class, 
                                   0,
                                   test_inputs[index:index+batch_size],
                                   meas, 
                                   [], #original_files[index:index+batch_size],
                                   [], #acts, 
                                   test_labels[index:index+batch_size])
    compute_sensitivity_scores(concept, 
                           rcvs_plus[concept][rop_class][0], 
                           model, 
                           layers[layer], 
                           rop_class, 
                           0,
                           test_inputs[index:],
                           meas, 
                           [],#original_files[index:],
                           [], #acts, 
                           test_labels[index:])'''
    compute_sensitivity_scores(concept, 
                                   rcvs_plus[concept][rop_class][0], 
                                   model, 
                                   layers[layer], 
                                   rop_class, 
                                   0,
                                   test_inputs[index:index+batch_size],
                                   meas, 
                                   [], #original_files[index:index+batch_size],
                                   [], #acts, 
                                   test_labels[index:index+batch_size])